# Improving Computer Vision Accuracy using Convolutions

In the previous lessons you saw how to do fashion recognition using a Deep Neural Network (DNN) containing three layers -- the input layer (in the shape of the data), the output layer (in the shape of the desired output) and a hidden layer. You experimented with the impact of different sized of hidden layer, number of training epochs etc on the final accuracy.

For convenience, here's the entire code again. Run it and take a note of the test accuracy that is printed out at the end. 

Your accuracy is probably about 89% on training and 87% on validation...not bad...But how do you make that even better? One way is to use something called Convolutions. I'm not going to details on Convolutions here, but the ultimate concept is that they narrow down the content of the image to focus on specific, distinct, details. 

If you've ever done image processing using a filter (like this: https://en.wikipedia.org/wiki/Kernel_(image_processing)) then convolutions will look very familiar.

In short, you take an array (usually 3x3 or 5x5) and pass it over the image. By changing the underlying pixels based on the formula within that matrix, you can do things like edge detection. So, for example, if you look at the above link, you'll see a 3x3 that is defined for edge detection where the middle cell is 8, and all of its neighbors are -1. In this case, for each pixel, you would multiply its value by 8, then subtract the value of each neighbor. Do this for every pixel, and you'll end up with a new image that has the edges enhanced.

This is perfect for computer vision, because often it's features that can get highlighted like this that distinguish one item for another, and the amount of information needed is then much less...because you'll just train on the highlighted features.

That's the concept of Convolutional Neural Networks. Add some layers to do convolution before you have the dense layers, and then the information going to the dense layers is more focussed, and possibly more accurate.

Run the below code -- this is the same neural network as earlier, but this time with Convolutional layers added first. It will take longer, but look at the impact on the accuracy:

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss = model.evaluate(test_images, test_labels)


It's likely gone up to about 93% on the training data and 91% on the validation data. 

That's significant, and a step in the right direction!

Try running it for more epochs -- say about 20, and explore the results! But while the results might seem really good, the validation results may actually go down, due to something called 'overfitting' which will be discussed later. 

(In a nutshell, 'overfitting' occurs when the network learns the data from the training set really well, but it's too specialised to only that data, and as a result is less effective at seeing *other* data. For example, if all your life you only saw red shoes, then when you see a red shoe you would be very good at identifying it, but blue suade shoes might confuse you...and you know you should never mess with my blue suede shoes.)

Then, look at the code again, and see, step by step how the Convolutions were built:

Step 1 is to gather the data. You'll notice that there's a bit of a change here in that the training data needed to be reshaped. That's because the first convolution expects a single tensor containing everything, so instead of 60,000 28x28x1 items in a list, we have a single 4D list that is 60,000x28x28x1, and the same for the test images. If you don't do this, you'll get an error when training as the Convolutions do not recognize the shape. 



```
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
```



Next is to define your model. Now instead of the input layer at the top, you're going to add a Convolution. The parameters are:

1. The number of convolutions you want to generate. Purely arbitrary, but good to start with something in the order of 32
2. The size of the Convolution, in this case a 3x3 grid
3. The activation function to use -- in this case we'll use relu, which you might recall is the equivalent of returning x when x>0, else returning 0
4. In the first layer, the shape of the input data.

You'll follow the Convolution with a MaxPooling layer which is then designed to compress the image, while maintaining the content of the features that were highlighted by the convlution. By specifying (2,2) for the MaxPooling, the effect is to quarter the size of the image. Without going into too much detail here, the idea is that it creates a 2x2 array of pixels, and picks the biggest one, thus turning 4 pixels into 1. It repeats this across the image, and in so doing halves the number of horizontal, and halves the number of vertical pixels, effectively reducing the image by 25%.

You can call model.summary() to see the size and shape of the network, and you'll notice that after every MaxPooling layer, the image size is reduced in this way. 


```
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
```



Add another convolution



```
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2)
```



In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

Now flatten the output. After this you'll just have the same DNN structure as the non convolutional version

```
  tf.keras.layers.Flatten(),
```



The same 128 dense layers, and 10 output layers as in the pre-convolution example:



```
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
```



Now compile the model, call the fit method to do the training, and evaluate the loss and accuracy from the test set.



```
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)
```




# Visualizing the Convolutions and Pooling

This code will show us the convolutions graphically. The print (test_labels[;100]) shows us the first 100 labels in the test set, and you can see that the ones at index 0, index 23 and index 28 are all the same value (9). They're all shoes. Let's take a look at the result of running the convolution on each, and you'll begin to see common features between them emerge. Now, when the DNN is training on that data, it's working with a lot less, and it's perhaps finding a commonality between shoes based on this convolution/pooling combination.

In [ ]:
print(test_labels[:100])

In [ ]:
import matplotlib.pyplot as plt
f, axarr = plt.subplots(3,4)
FIRST_IMAGE=0
SECOND_IMAGE=7
THIRD_IMAGE=26
CONVOLUTION_NUMBER = 1
from tensorflow.keras import models
layer_outputs = [layer.output for layer in model.layers]
activation_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
for x in range(0,4):
  f1 = activation_model.predict(test_images[FIRST_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[0,x].imshow(f1[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[0,x].grid(False)
  f2 = activation_model.predict(test_images[SECOND_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[1,x].imshow(f2[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[1,x].grid(False)
  f3 = activation_model.predict(test_images[THIRD_IMAGE].reshape(1, 28, 28, 1))[x]
  axarr[2,x].imshow(f3[0, : , :, CONVOLUTION_NUMBER], cmap='inferno')
  axarr[2,x].grid(False)

# EXERCISES

1. Try editing the convolutions. Change the 32s to either 16 or 64. What impact will this have on accuracy and/or training time.

2. Remove the final Convolution. What impact will this have on accuracy or training time?

3. How about adding more Convolutions? What impact do you think this will have? Experiment with it.

4. Remove all Convolutions but the first. What impact do you think this will have? Experiment with it. 

5. In the previous lesson you implemented a callback to check on the loss function and to cancel training once it hit a certain amount. See if you can implement that here!

In [ ]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

# EXERCISES

1. Try editing the convolutions. Change the 32s to either 16 or 64. What impact will this have on accuracy and/or training time.

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images /255.0

# CREATE THE MODEL
model = tf.keras.models.Sequential([
    # FIRST CONVOLUTIONAL LAYER
    # number of convolutions = 32 (arbitary) the size of the convolution is 3x3
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    # maxPooling is 2x2 array of pixels and picks the biggest one aka turns 4 pixels into 1
    tf.keras.layers.MaxPooling2D(2,2),
    # SECOND CONVOLUTIONAL LAYER
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # FLATTEN THE OUTPUT
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# COMPILE THE MODEL
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

#### answer

###### 16 Convolutions = 91% accuracy
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 233us/sample - loss: 0.4589 - accuracy: 0.8328

Epoch 2/10
60000/60000 [==============================] - 11s 179us/sample - loss: 0.3088 - accuracy: 0.8857

Epoch 3/10
60000/60000 [==============================] - 11s 180us/sample - loss: 0.2629 - accuracy: 0.9033

Epoch 4/10
60000/60000 [==============================] - 11s 180us/sample - loss: 0.2295 - accuracy: 0.9155

Epoch 5/10
60000/60000 [==============================] - 11s 179us/sample - loss: 0.2047 - accuracy: 0.9238

Epoch 6/10
60000/60000 [==============================] - 11s 178us/sample - loss: 0.1807 - accuracy: 0.9330

Epoch 7/10
60000/60000 [==============================] - 11s 180us/sample - loss: 0.1611 - accuracy: 0.9391

Epoch 8/10
60000/60000 [==============================] - 11s 176us/sample - loss: 0.1427 - accuracy: 0.9466

Epoch 9/10
60000/60000 [==============================] - 10s 174us/sample - loss: 0.1260 - accuracy: 0.9527

Epoch 10/10
60000/60000 [==============================] - 11s 179us/sample - loss: 0.1125 - accuracy: 0.9576

10000/10000 [==============================] - 2s 155us/sample - loss: 0.2977 - accuracy: 0.9128

0.9128


###### 64 Convolutions = 92% accucy in test

Train on 60000 samples

Epoch 1/10
60000/60000 [==============================] - 12s 198us/sample - loss: 0.4336 - accuracy: 0.8444

Epoch 2/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.2910 - accuracy: 0.8936

Epoch 3/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.2463 - accuracy: 0.9093

Epoch 4/10
60000/60000 [==============================] - 11s 180us/sample - loss: 0.2154 - accuracy: 0.9198

Epoch 5/10
60000/60000 [==============================] - ETA: 0s - loss: 0.1884 - accuracy: 0.92 - 11s 179us/sample - loss: 0.1886 - 
accuracy: 0.9293

Epoch 6/10
60000/60000 [==============================] - 11s 183us/sample - loss: 0.1669 - accuracy: 0.9377

Epoch 7/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.1459 - accuracy: 0.9455

Epoch 8/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.1273 - accuracy: 0.9523

Epoch 9/10
60000/60000 [==============================] - 11s 184us/sample - loss: 0.1105 - accuracy: 0.9583

Epoch 10/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.0994 - accuracy: 0.9628

10000/10000 [==============================] - 2s 167us/sample - loss: 0.2888 - accuracy: 0.9193

0.9193

##### The number of convolutions impacts the time (the more convoultions the longer the time frame to process)
###### The more convolutions the higher the accuracy and the lower the loss

# EXERCISES
2. Remove the final Convolution. What impact will this have on accuracy or training time?

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images /255.0

# CREATE THE MODEL
model = tf.keras.models.Sequential([
    # FIRST CONVOLUTIONAL LAYER
    # number of convolutions = 32 (arbitary) the size of the convolution is 3x3
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    # maxPooling is 2x2 array of pixels and picks the biggest one aka turns 4 pixels into 1
    tf.keras.layers.MaxPooling2D(2,2),
    # FLATTEN THE OUTPUT
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# COMPILE THE MODEL
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

#### answer
Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 181us/sample - loss: 0.3908 - accuracy: 0.8617
Epoch 2/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.2645 - accuracy: 0.9039
Epoch 3/10
60000/60000 [==============================] - 10s 164us/sample - loss: 0.2196 - accuracy: 0.9190
Epoch 4/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.1862 - accuracy: 0.9309
Epoch 5/10
60000/60000 [==============================] - 10s 163us/sample - loss: 0.1589 - accuracy: 0.9400
Epoch 6/10
60000/60000 [==============================] - 10s 160us/sample - loss: 0.1370 - accuracy: 0.9483
Epoch 7/10
60000/60000 [==============================] - 10s 163us/sample - loss: 0.1146 - accuracy: 0.9575
Epoch 8/10
60000/60000 [==============================] - 9s 158us/sample - loss: 0.0970 - accuracy: 0.9638
Epoch 9/10
60000/60000 [==============================] - 9s 155us/sample - loss: 0.0824 - accuracy: 0.9695
Epoch 10/10
60000/60000 [==============================] - 9s 152us/sample - loss: 0.0694 - accuracy: 0.9744
10000/10000 [==============================] - 2s 162us/sample - loss: 0.3411 - accuracy: 0.9143
0.9143


the training time is deccreased by approx 1-2 seconds

the accuracy is reduced to 91%

# EXERCISES

3. How about adding more Convolutions? What impact do you think this will have? Experiment with it.

In [ ]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images /255.0

# CREATE THE MODEL
model = tf.keras.models.Sequential([
    # FIRST CONVOLUTIONAL LAYER
    # number of convolutions = 32 (arbitary) the size of the convolution is 3x3
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    # maxPooling is 2x2 array of pixels and picks the biggest one aka turns 4 pixels into 1
    tf.keras.layers.MaxPooling2D(2,2),
    # SECOND CONVOLUTIONAL LAYER
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # THIRD CONVOLUTIONAL LAYER
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # FLATTEN THE OUTPUT
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# COMPILE THE MODEL
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

#### answer
adding another convolutional layer overtraining the model and reduced the accuracy of the test

# EXERCISES

4. Remove all Convolutions but the first. What impact do you think this will have? Experiment with it. 

#### answer


# EXERCISES

5. In the previous lesson you implemented a callback to check on the loss function and to cancel training once it hit a certain amount. See if you can implement that here!

In [8]:
import tensorflow as tf

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss') < 0.40 ):
            print('\nReached 60% accuracy -stopping training')
            self.model.stop_training=True
                                       
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images = test_images /255.0

callbacks=myCallback()

# CREATE THE MODEL
model = tf.keras.models.Sequential([
    # FIRST CONVOLUTIONAL LAYER
    # number of convolutions = 32 (arbitary) the size of the convolution is 3x3
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    # maxPooling is 2x2 array of pixels and picks the biggest one aka turns 4 pixels into 1
    tf.keras.layers.MaxPooling2D(2,2),
    # SECOND CONVOLUTIONAL LAYER
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # THIRD CONVOLUTIONAL LAYER
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # FLATTEN THE OUTPUT
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# COMPILE THE MODEL
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 13s 218us/sample - loss: 0.5906 - accuracy: 0.7841
Epoch 2/10
60000/60000 [==============================] - 11s 176us/sample - loss: 0.4063 - accuracy: 0.8497
Epoch 3/10
59392/60000 [============================>.] - ETA: 0s - loss: 0.3500 - accuracy: 0.8711
Reached 60% accuracy -stopping training
10000/10000 [==============================] - 1s 75us/sample - loss: 0.3702 - accuracy: 0.8649
0.8649


#### answer